In [ ]:
pip install scikit-optimize

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, LSTM, Dense, Flatten, Activation, RepeatVector, Permute, Multiply, Lambda, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.base import BaseEstimator, RegressorMixin
from skopt import BayesSearchCV
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt



In [ ]:
# Load and combine data from multiple CSV files
csv_file_paths = [
    'cold_season_NITJ_data.csv',
    'monsoon_season_NITJ_data.csv',
    'post_monsoon_NITJ_data.csv',
    'summer_season_NITJ_data.csv'
]

dfs = [pd.read_csv(path, parse_dates=['Date'], index_col='Date') for path in csv_file_paths]
data = pd.concat(dfs)
data.head()

,GHI,Clearsky GHI,Temperature,Pressure,Relative Humidity,Solar Zenith Angle,Wind Speed,Wind Direction,Dew Point,Precipitable Water,Clearness Index,Season
Date,,,,,,,,,,,,
2000-01-01 08:00:00,118,134,14,990,18.48,79.26,0.5,22.7,-9,0.515,0.880597,Cold Season
2000-01-01 09:00:00,262,311,18,990,8.87,69.54,0.4,351.4,-15,0.514,0.842444,Cold Season
2000-01-01 10:00:00,375,460,22,990,6.73,61.63,0.8,300.1,-15,0.514,0.815217,Cold Season
2000-01-01 11:00:00,524,557,25,990,5.81,56.33,1.4,293.9,-15,0.515,0.940754,Cold Season
2000-01-01 12:00:00,584,592,26,990,5.58,54.40,1.6,292.8,-15,0.516,0.986486,Cold Season


In [ ]:
# Define input features (X) and target variable (y)
X = data[['Clearsky GHI', 'Temperature', 'Pressure', 'Relative Humidity',
          'Solar Zenith Angle', 'Wind Speed', 'Wind Direction',
          'Dew Point', 'Precipitable Water', 'Clearness Index']]
y = data['GHI']


gice window size and for making predictions

Give code for minmax scaler

In [ ]:
# Train-test split for the first model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=2/3, random_state=42)
X_train = np.expand_dims(X_train, axis=-1)
X_val = np.expand_dims(X_val, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)


Add code for reshapping the input variables

Give the code Defining the classs for attention model

In [ ]:
# Define the attention model
def build_model(learning_rate, lstm_units):
    inputs = Input(shape=(X_train.shape[1], 1))
    conv = Conv1D(filters=64, kernel_size=3, activation='relu')(inputs)
    pool = MaxPooling1D(pool_size=2)(conv)
    lstm = LSTM(lstm_units, return_sequences=True)(pool)
    attention = Dense(1, activation='tanh')(lstm)
    attention = Flatten()(attention)
    attention = Activation('softmax')(attention)
    attention = RepeatVector(lstm_units)(attention)
    attention = Permute([2, 1])(attention)
    sent_representation = Multiply()([lstm, attention])
    sent_representation = Lambda(lambda xin: K.sum(xin, axis=-2), output_shape=(lstm_units,))(sent_representation)
    outputs = Dense(1)(sent_representation)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse', metrics=['mae'])
    return model


define the param space
the give the code for bayesian optimization

In [ ]:
# Train the initial model with early stopping
model = build_model(learning_rate=0.001, lstm_units=64, filters=64)
early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
model.fit(X_train, y_train, epochs=1, validation_data=(X_val, y_val), callbacks=[early_stop], verbose=1)


Epoch 1/50
1405/1405 [==============================] - 16s 9ms/step - loss: 210299.3438 - mae: 374.2261 - val_loss: 182021.1250 - val_mae: 343.8634
Epoch 2/50
1405/1405 [==============================] - 13s 9ms/step - loss: 155532.2656 - mae: 314.9175 - val_loss: 136298.7812 - val_mae: 295.0695
Epoch 3/50
1405/1405 [==============================] - 13s 9ms/step - loss: 117433.6250 - mae: 274.2988 - val_loss: 103829.2734 - val_mae: 260.2697
Epoch 4/50
1405/1405 [==============================] - 14s 10ms/step - loss: 89025.2812 - mae: 239.5787 - val_loss: 77317.4219 - val_mae: 222.8641
Epoch 5/50
1405/1405 [==============================] - 13s 9ms/step - loss: 61670.5352 - mae: 187.1910 - val_loss: 50631.4883 - val_mae: 162.5909
Epoch 6/50
1405/1405 [==============================] - 12s 8ms/step - loss: 42153.3750 - mae: 146.7676 - val_loss: 36210.0547 - val_mae: 135.1209
Epoch 7/50
1405/1405 [==============================] - 11s 8ms/step - loss: 30250.2559 - mae: 122.9234 - val_l

In [ ]:
# Create a wrapper class for Keras model
class KerasRegressorWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, create_model, params):
        self.create_model = create_model
        self.params = params
        self.model = None

    def fit(self, X, y):
        self.model = self.create_model(**self.params)
        self.model.fit(X, y, epochs=50, batch_size=32, verbose=0)
        return self

    def predict(self, X):
        return self.model.predict(X)

    def score(self, X, y):
        y_pred = self.predict(X)
        return r2_score(y, y_pred)

below code needs to be defind above

In [ ]:
# Define the create_model function for the bayesian optimization model
def create_model(filters, kernel_size, pool_size, lstm_units, dense_units, dropout_rate, learning_rate):
    inputs = Input(shape=(X_train.shape[1], 1))
    conv = Conv1D(filters=filters, kernel_size=kernel_size, activation='relu')(inputs)
    pool = MaxPooling1D(pool_size=pool_size)(conv)
    lstm = LSTM(lstm_units, return_sequences=True)(pool)
    attention = Dense(1, activation='tanh')(lstm)
    attention = Flatten()(attention)
    attention = Activation('softmax')(attention)
    attention = RepeatVector(lstm_units)(attention)
    attention = Permute([2, 1])(attention)
    sent_representation = Multiply()([lstm, attention])
    sent_representation = Lambda(lambda xin: K.sum(xin, axis=-2), output_shape=(lstm_units,))(sent_representation)
    dense = Dense(dense_units, activation='relu')(sent_representation)
    dense = Dropout(dropout_rate)(dense)
    outputs = Dense(1)(dense)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse', metrics=['mae'])
    return model


In [ ]:
# Define the hyperparameter search space
param_space = {
    'filters': (16, 32, 64),
    'kernel_size': (3, 5),
    'pool_size': (2, 4),
    'lstm_units': (64, 128, 256),
    'dense_units': [32, 64, 128],
    'dropout_rate': (0.0, 0.5),
    'learning_rate': (0.001, 0.01, 0.1)
}



In [ ]:
# Perform Bayesian optimization for the second model
bayes_search = BayesSearchCV(estimator=KerasRegressorWrapper(create_model, params={}), search_spaces=param_space, n_iter=20, cv=3, random_state=42)
bayes_search.fit(X_train, y_train)


ValueError: Invalid parameter 'dense_units' for estimator KerasRegressorWrapper(create_model=<function create_model at 0x7a729c603d00>,
                      params={}). Valid parameters are: ['create_model', 'params'].

In [ ]:
# Evaluate the best model on test set for the second model
best_model = bayes_search.best_estimator_
y_pred_test = best_model.predict(X_test)
r2_test = best_model.score(X_test, y_test)
rmse_test = mean_squared_error(y_test, y_pred_test, squared=False)
mae_test = mean_absolute_error(y_test, y_pred_test)
mse_test = mean_squared_error(y_test, y_pred_test)


# Evaluate the best model on validation set for the second model
r2_val = best_model.score(X_val.values.reshape((X_val.shape[0], X_val.shape[1], 1)), y_val)
rmse_val = mean_squared_error(y_val, best_model.predict(X_val.values.reshape((X_val.shape[0], X_val.shape[1], 1))), squared=False)
mae_val = mean_absolute_error(y_val, best_model.predict(X_val.values.reshape((X_val.shape[0], X_val.shape[1], 1))))
mse_val = mean_squared_error(y_val, best_model.predict(X_val.values.reshape((X_val.shape[0], X_val.shape[1], 1))))


print('Test set:')
print('R2 Score:', r2_test)
print('Root Mean Squared Error (RMSE):', rmse_test)
print('Mean Absolute Error (MAE):', mae_test)
print('Mean Squared Error (MSE):', mse_test)

print('\nValidation set:')
print('R2 Score:', r2_val)
print('Root Mean Squared Error (RMSE):', rmse_val)
print('Mean Absolute Error (MAE):', mae_val)
print('Mean Squared Error (MSE):', mse_val)


AttributeError: 'BayesSearchCV' object has no attribute 'best_estimator_'

In [ ]:

# Plotting the actual vs. predicted values for the test set
y_pred_test = best_model.predict(X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1)))

plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_test, color='blue')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], '--', color='red')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Actual vs. Predicted Values (Test Set)')
plt.grid(True)
plt.show()

# Plotting the actual vs. predicted values for the validation set
y_pred_val = best_model.predict(X_val.values.reshape((X_val.shape[0], X_val.shape[1], 1)))

plt.figure(figsize=(10, 6))
plt.scatter(y_val, y_pred_val, color='blue')
plt.plot([min(y_val), max(y_val)], [min(y_val), max(y_val)], '--', color='red')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Actual vs. Predicted Values (Validation Set)')
plt.grid(True)
plt.show()


NameError: name 'best_model' is not defined